Majority vote from team's CV

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import glob

from tqdm import tqdm
import seaborn as sns
from collections import Counter

import cv2

In [2]:
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

In [3]:
from collections import Counter
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

In [4]:
from collections import defaultdict

In [5]:
import bz2
import pickle
import _pickle as cPickle

def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        cPickle.dump(data, f)
        
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [6]:
#exps = !ls ../input/whale-21-3-2022/ | grep pbz2 | grep fold4
#exps

In [7]:
FOLD=1

In [8]:
exps_all = [
    [
         'exp16v6Augmix_fold0.pbz2',
         'exp16v4sampler_fold0.pbz2',
         'exp16v6AugMixNew4_fold0.pbz2',
         'exp16v6AugMixNew4_fold0_tta.pbz2',
         'exp16v6AugmixB3_fold0.pbz2',
         'exp16v6AugmixB4Step2_fold0.pbz2',
         'exp16v6AugmixB4Step2_fold0_remove_background.pbz2',
         'exp16v6AugmixB4_fold0.pbz2',
         'exp16v6AugmixB4_fold0_remove_background.pbz2',
         'exp16v6AugmixFullB4_fold0.pbz2',
         'exp16v6sampler_fold0.pbz2',
         'exp21b6_fold0.pbz2',
         'exp21b6_fold0_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold1.pbz2',
         'exp16v6AugMixNew4_fold1.pbz2',
         'exp16v6AugMixNew4_fold1_tta.pbz2',
         'exp16v6AugmixB3_fold1.pbz2',
         'exp16v6AugmixB4Step2_fold1.pbz2',
         'exp16v6AugmixB4Step2_fold1_remove_background.pbz2',
         'exp16v6AugmixB4_fold1.pbz2',
         'exp21b6_fold1_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold2.pbz2',
         'exp16v6AugMixNew4_fold2.pbz2',
         'exp16v6AugMixNew4_fold2_tta.pbz2',
         'exp16v6AugmixB3_fold2.pbz2',
         'exp16v6AugmixB4Full_fold2.pbz2',
         'exp16v6AugmixB4Step2_fold2.pbz2',
         'exp16v6AugmixB4_fold2.pbz2',
         'exp16v6AugmixB4_fold2_remove_background.pbz2',
         'exp21b6_fold2_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold3.pbz2',
         'exp16v6AugMix1536_fold3.pbz2',
         'exp16v6AugmixB3_fold3.pbz2',
         'exp16v6AugmixB4Full_fold3.pbz2',
         'exp16v6AugmixB4Step2_fold3.pbz2',
         'exp16v6AugmixB4_fold3_remove_background.pbz2',
         'exp21b6_fold3_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold4.pbz2',
         'exp16v6AugMix1536_fold4.pbz2',
         'exp16v6AugmixB3_fold4.pbz2',
         'exp16v6AugmixB4Step2_fold4.pbz2',
         'exp16v6AugmixB4_fold4_remove_background.pbz2',
    ]
]

In [9]:
exps = exps_all[FOLD]

In [10]:
fold2 = pd.read_csv("/kaggle/happywhale/input/whale-21-3-2022/fold2.csv")
fold2.shape

(51033, 8)

In [11]:
valid = fold2.query("fold == @FOLD").reset_index()
train = fold2.query("fold != @FOLD").reset_index()

In [12]:
valid_ind = (fold2.fold == FOLD).values
train_ind = (fold2.fold != FOLD).values

In [13]:
train_inddividual_id_lis = train["individual_id"].values
valid_inddividual_id_lis = valid["individual_id"].values

def get_result_each(files):
    DIRNAME ="../input/whale-21-3-2022/"
      
    ret = []
        
    for fname in files:
        print(fname)
        data = decompress_pickle(DIRNAME + fname  )
        
        train_emb = data["emb_val"]#[new_index,:]
        
        train_emb=normalize(train_emb)
        
        valid_preds = train_emb[valid_ind]
        train_preds = train_emb[train_ind]
        
        
        distances = pairwise_distances(valid_preds,train_preds)
                
        K=1000
        predicted_positions = np.argpartition(distances, K, axis=1)[:, :K]

        all_pred_lis = []
        for i in tqdm(range(predicted_positions.shape[0])):
            nearest = [(train_inddividual_id_lis[j], distances[i, j]) for j in predicted_positions[i]]
            nearest.sort(key=lambda x: x[1])

            prediction = [index_id for index_id, d in nearest]

            done = set()
            pred_list = []
            for pred_id in prediction:
                if pred_id in done:
                    continue
                done.add(pred_id)
                pred_list.append(pred_id)
                if len(pred_list)==20:
                    break

            all_pred_lis.append(pred_list)
        
        ret.append(all_pred_lis)
        
    return ret
    

### get result for each model

In [14]:
ret = get_result_each(exps)

exp16v6Augmix_fold1.pbz2


100%|██████████| 8319/8319 [00:07<00:00, 1101.15it/s]


exp16v6AugMixNew4_fold1.pbz2


100%|██████████| 8319/8319 [00:07<00:00, 1111.50it/s]


exp16v6AugMixNew4_fold1_tta.pbz2


...

100%|██████████| 8319/8319 [00:07<00:00, 1102.38it/s]


exp16v6AugmixB4_fold1.pbz2


100%|██████████| 8319/8319 [00:07<00:00, 1110.78it/s]


exp21b6_fold1_remove_background.pbz2


100%|██████████| 8319/8319 [00:07<00:00, 1113.48it/s]


In [15]:
len(ret),len(ret[0]),len(ret[0][0])

(8, 8319, 20)

In [16]:
def get_score(weights):
    
    #train_emb
    vote_result = defaultdict(lambda:defaultdict(int))
    
    for preds, w in zip(ret, weights):
        for i in range(len(preds)):
            
            for j,p in enumerate(preds[i]):
                vote_result[i][p] += (w / (j+1))
        
        
    score_lis = []
    pred_ids = []

    for i in range(len(ret[0])):
        pred_list = np.array(sorted(vote_result[i].items(), key=lambda item: item[1])[::-1])[:5,0].tolist()

        if valid_inddividual_id_lis[i] in pred_list:
            s = 1/(pred_list.index(valid_inddividual_id_lis[i]) + 1)
            score_lis.append(s)
        else:
            score_lis.append(0)
    
    return np.mean(score_lis)

In [17]:
# un-weighted result
get_score([1] * len(exps))

0.85759907040109

optimize by optuna

In [18]:
import optuna

In [19]:
def objective_wrapper(exps):
    def objective(trial):

        weights = [1]
        for i in range(len(exps)-1):
            w=trial.suggest_uniform(exps[i+1], 0, 4)
            weights.append(w)
        
        
        score_lis = get_score(weights)
        score = np.mean(score_lis)
        return score
    
    return objective


In [20]:
%%time

study = optuna.create_study(direction='maximize')
study.optimize(objective_wrapper(exps), timeout=300)

[I 2022-04-17 07:36:02,168] A new study created in memory with name: no-name-4134d034-3d30-49e8-a7c1-357bba0d4d85
[I 2022-04-17 07:36:03,308] Trial 0 finished with value: 0.8615799174580279 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 3.2891669351629576, 'exp16v6AugMixNew4_fold1_tta.pbz2': 2.2331749519119675, 'exp16v6AugmixB3_fold1.pbz2': 3.3824665369159925, 'exp16v6AugmixB4Step2_fold1.pbz2': 2.106206410728921, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 0.14880076291902133, 'exp16v6AugmixB4_fold1.pbz2': 1.2065625084409888, 'exp21b6_fold1_remove_background.pbz2': 3.264666861406923}. Best is trial 0 with value: 0.8615799174580279.
[I 2022-04-17 07:36:04,449] Trial 1 finished with value: 0.8540970469207037 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 1.189756765964697, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.2670995660730555, 'exp16v6AugmixB3_fold1.pbz2': 0.6141279153728383, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.7324607481059848, 'exp16v6AugmixB4Step2_fold1_remove_backgroun

[I 2022-04-17 07:36:20,692] Trial 15 finished with value: 0.8580839043154225 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 3.952977606394085, 'exp16v6AugMixNew4_fold1_tta.pbz2': 0.08473520832891546, 'exp16v6AugmixB3_fold1.pbz2': 0.10093545840104168, 'exp16v6AugmixB4Step2_fold1.pbz2': 2.4572707472842703, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 1.435549218599175, 'exp16v6AugmixB4_fold1.pbz2': 2.4962742286438315, 'exp21b6_fold1_remove_background.pbz2': 0.7804261348184993}. Best is trial 11 with value: 0.8685458989461874.
[I 2022-04-17 07:36:21,844] Trial 16 finished with value: 0.8655226990423528 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 2.627740689588543, 'exp16v6AugMixNew4_fold1_tta.pbz2': 2.518555687115726, 'exp16v6AugmixB3_fold1.pbz2': 1.6934812380398632, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.9578219875997748, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 2.0042754911859513, 'exp16v6AugmixB4_fold1.pbz2': 1.6956597409106204, 'exp21b6_fold1_remove_background

[I 2022-04-17 07:36:38,149] Trial 30 finished with value: 0.8580217974916857 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 1.9140546419960576, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.0562592617247863, 'exp16v6AugmixB3_fold1.pbz2': 0.4070301064146832, 'exp16v6AugmixB4Step2_fold1.pbz2': 2.754764768789325, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 0.7295853969617598, 'exp16v6AugmixB4_fold1.pbz2': 0.2881687126925885, 'exp21b6_fold1_remove_background.pbz2': 2.063278583831114}. Best is trial 19 with value: 0.8691108706975998.
[I 2022-04-17 07:36:39,308] Trial 31 finished with value: 0.8688764675241415 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 3.6507142848271563, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.8502122504008873, 'exp16v6AugmixB3_fold1.pbz2': 0.7722689603308774, 'exp16v6AugmixB4Step2_fold1.pbz2': 1.7710188557401898, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 1.0624912446428543, 'exp16v6AugmixB4_fold1.pbz2': 1.075713590990104, 'exp21b6_fold1_remove_background.

[I 2022-04-17 07:36:55,782] Trial 45 finished with value: 0.8623312096806507 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 2.954059582117748, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.3877480609981048, 'exp16v6AugmixB3_fold1.pbz2': 0.518056636894625, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.038434367997768115, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 3.9027445202325084, 'exp16v6AugmixB4_fold1.pbz2': 1.497861066704624, 'exp21b6_fold1_remove_background.pbz2': 1.572370960435502}. Best is trial 41 with value: 0.8702428176463517.
[I 2022-04-17 07:36:56,982] Trial 46 finished with value: 0.8606262771967785 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 2.5438478951433154, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.6448229727944692, 'exp16v6AugmixB3_fold1.pbz2': 3.2169245427687754, 'exp16v6AugmixB4Step2_fold1.pbz2': 1.318569437173458, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 0.2691881982190293, 'exp16v6AugmixB4_fold1.pbz2': 1.2101569467355102, 'exp21b6_fold1_remove_background.

[I 2022-04-17 07:37:13,381] Trial 60 finished with value: 0.8610229594903234 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 3.697237715797732, 'exp16v6AugMixNew4_fold1_tta.pbz2': 0.23763813241367326, 'exp16v6AugmixB3_fold1.pbz2': 1.5718675015301247, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.3377908424680553, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 0.6568228332113574, 'exp16v6AugmixB4_fold1.pbz2': 3.059571984583323, 'exp21b6_fold1_remove_background.pbz2': 0.37813078507958164}. Best is trial 41 with value: 0.8702428176463517.
[I 2022-04-17 07:37:14,544] Trial 61 finished with value: 0.869068798333133 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 3.4831847944018204, 'exp16v6AugMixNew4_fold1_tta.pbz2': 0.9705797466506391, 'exp16v6AugmixB3_fold1.pbz2': 0.5270200723339247, 'exp16v6AugmixB4Step2_fold1.pbz2': 1.035163885659886, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 0.8688027305603725, 'exp16v6AugmixB4_fold1.pbz2': 0.9015504878397039, 'exp21b6_fold1_remove_background

...

[I 2022-04-17 07:40:09,565] Trial 210 finished with value: 0.8671174420002403 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 2.485490437974025, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.6395451997289097, 'exp16v6AugmixB3_fold1.pbz2': 1.4630599318115687, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.7919493452678434, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 1.1634901433804448, 'exp16v6AugmixB4_fold1.pbz2': 0.479945168716592, 'exp21b6_fold1_remove_background.pbz2': 0.9608867799670482}. Best is trial 191 with value: 0.8704071002123653.
[I 2022-04-17 07:40:10,729] Trial 211 finished with value: 0.8699162559602516 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 2.4733226568237914, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.5706164591421068, 'exp16v6AugmixB3_fold1.pbz2': 0.07726852408101503, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.3533846292539134, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 1.061735068235126, 'exp16v6AugmixB4_fold1.pbz2': 0.6178268221328637, 'exp21b6_fold1_remove_backgro

[I 2022-04-17 07:40:27,249] Trial 225 finished with value: 0.8703189485915774 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 2.6827576187298914, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.7910405544256642, 'exp16v6AugmixB3_fold1.pbz2': 0.13793143676648453, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.5857822284708992, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 1.4636267918083616, 'exp16v6AugmixB4_fold1.pbz2': 0.4650242747004857, 'exp21b6_fold1_remove_background.pbz2': 1.314326437521221}. Best is trial 191 with value: 0.8704071002123653.
[I 2022-04-17 07:40:28,425] Trial 226 finished with value: 0.8699322835276676 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 2.666703823948184, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.8975513863773092, 'exp16v6AugmixB3_fold1.pbz2': 0.14542791417139012, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.5300960272497276, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 1.44031394772534, 'exp16v6AugmixB4_fold1.pbz2': 0.6155620529566783, 'exp21b6_fold1_remove_backgro

[I 2022-04-17 07:40:44,851] Trial 240 finished with value: 0.8696337700845453 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 2.6584081386463017, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.8605636189503048, 'exp16v6AugmixB3_fold1.pbz2': 0.18877436442077167, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.6873794402877678, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 1.3485854171167253, 'exp16v6AugmixB4_fold1.pbz2': 0.6641575759958277, 'exp21b6_fold1_remove_background.pbz2': 1.5388321213739033}. Best is trial 191 with value: 0.8704071002123653.
[I 2022-04-17 07:40:46,043] Trial 241 finished with value: 0.8702688624434026 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 2.567606512637053, 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.6916294912873417, 'exp16v6AugmixB3_fold1.pbz2': 0.08045501934483262, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.5800748093116282, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 1.3307196916311104, 'exp16v6AugmixB4_fold1.pbz2': 0.3522131007152093, 'exp21b6_fold1_remove_back

[I 2022-04-17 07:41:02,849] Trial 255 finished with value: 0.8699903834595505 and parameters: {'exp16v6AugMixNew4_fold1.pbz2': 2.827130795892987, 'exp16v6AugMixNew4_fold1_tta.pbz2': 2.028425226755751, 'exp16v6AugmixB3_fold1.pbz2': 0.003980748849481299, 'exp16v6AugmixB4Step2_fold1.pbz2': 0.4879588980355316, 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 1.5951343406732421, 'exp16v6AugmixB4_fold1.pbz2': 0.6189132166350798, 'exp21b6_fold1_remove_background.pbz2': 1.2999686560028298}. Best is trial 245 with value: 0.8704551829146131.


CPU times: user 5min, sys: 1.27 s, total: 5min 2s
Wall time: 5min


In [21]:
study.trials_dataframe().sort_values("value")

,number,value,datetime_start,datetime_complete,duration,params_exp16v6AugMixNew4_fold1.pbz2,params_exp16v6AugMixNew4_fold1_tta.pbz2,params_exp16v6AugmixB3_fold1.pbz2,params_exp16v6AugmixB4Step2_fold1.pbz2,params_exp16v6AugmixB4Step2_fold1_remove_background.pbz2,params_exp16v6AugmixB4_fold1.pbz2,params_exp21b6_fold1_remove_background.pbz2,state
99,99,0.841471,2022-04-17 07:37:57.750392,2022-04-17 07:37:58.905284,0 days 00:00:01.154892,0.024390,0.990500,2.461089,0.513452,0.318804,1.321254,1.545943,COMPLETE
2,2,0.842758,2022-04-17 07:36:04.451019,2022-04-17 07:36:05.590919,0 days 00:00:01.139900,1.638985,0.343407,3.047685,0.120724,1.623954,3.330185,3.892602,COMPLETE
79,79,0.845811,2022-04-17 07:37:34.338040,2022-04-17 07:37:35.496539,0 days 00:00:01.158499,0.868806,0.206655,0.084676,0.387060,0.891670,2.583445,0.807407,COMPLETE
20,20,0.847894,2022-04-17 07:36:25.343263,2022-04-17 07:36:26.524280,0 days 00:00:01.181017,0.266296,1.526601,2.379017,0.670873,0.399951,2.552271,1.732782,COMPLETE
8,8,0.849701,2022-04-17 07:36:11.314948,2022-04-17 07:36:12.461140,0 days 00:00:01.146192,0.225826,2.899206,2.506441,3.178273,0.935928,3.424852,3.094058,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,0.870369,2022-04-17 07:40:18.942682,2022-04-17 07:40:20.235943,0 days 00:00:01.293261,2.688012,1.891314,0.079614,0.601047,1.391657,0.502238,1.315679,COMPLETE
217,217,0.870405,2022-04-17 07:40:16.599122,2022-04-17 07:40:17.779876,0 days 00:00:01.180754,2.697159,1.820993,0.069946,0.583231,1.371528,0.525969,1.302725,COMPLETE
191,191,0.870407,2022-04-17 07:39:45.994001,2022-04-17 07:39:47.152344,0 days 00:00:01.158343,2.331028,1.712741,0.075095,0.497696,1.029562,0.787737,1.038836,COMPLETE
247,247,0.870407,2022-04-17 07:40:52.288780,2022-04-17 07:40:53.458263,0 days 00:00:01.169483,2.869236,1.781540,0.003107,0.485312,1.503573,0.660783,1.255732,COMPLETE


In [22]:
study.best_params

{'exp16v6AugMixNew4_fold1.pbz2': 2.68754947649628,
 'exp16v6AugMixNew4_fold1_tta.pbz2': 1.6919651463177248,
 'exp16v6AugmixB3_fold1.pbz2': 0.061478256930647394,
 'exp16v6AugmixB4Step2_fold1.pbz2': 0.504805001940402,
 'exp16v6AugmixB4Step2_fold1_remove_background.pbz2': 1.3077181638709203,
 'exp16v6AugmixB4_fold1.pbz2': 0.5794563909474894,
 'exp21b6_fold1_remove_background.pbz2': 1.2056540242971114}

In [23]:
best_weights = list(study.best_params.values())

In [24]:
# weighted result
get_score([1] + best_weights)

0.8704551829146131

In [25]:
# un-weighted result
get_score([1] * len(exps))

0.85759907040109